In [46]:
import numpy as np
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import string
import re
import gensim.models
import tempfile
from gensim.models import Word2Vec
import time

# Training our model

In [47]:
filter_stopwords = False
filter_pronouns = True

# Reading in our sentences from the scrape results
matthew = False
path = "C:/Users/laure/OneDrive/Documenten/Project 3.1 APG Files/"
filename = 'public_and_private_sentences_{}_filtered.csv'.format('all' if filter_stopwords and filter_pronouns else 'stopwords' if filter_stopwords else 'pronouns' if filter_pronouns else 'none')
if matthew:
    df = pd.read_csv('DATA/scrape_results/' + filename)
else:
    df = pd.read_csv(path + filename)


In [48]:
# Converting to list
corpus = df['sentences'].tolist()

In [49]:
# Break each sentence into list of individual words
sentences = []
for sentence in corpus:
    sentences.append(sentence.split(" "))

In [50]:
# Create custom word2vec model with given parameters (from internet)
custom_model = Word2Vec(window = 10, min_count = 5, workers = 20, negative = 5, sample = 1e-5)

# Build the vocab by giving the sentences
custom_model.build_vocab(sentences, progress_per=10000)

# Train the model on the inputted sentences, epoch = 50, higher epochs give better output but increase time
start = time.time()
custom_model.train(sentences, total_examples=custom_model.corpus_count, epochs=50, report_delay=1)
end = time.time()

print("Time elapsed (seconds): " + str(end - start))

Time elapsed (seconds): 17.416813373565674


In [51]:
# Save the model for re-use
filename = 'custom_model_{}_filtered.csv'.format('all' if filter_stopwords and filter_pronouns else 'stopwords' if filter_stopwords else 'pronouns' if filter_pronouns else 'none')
if matthew:
    custom_model.save('DATA/Data/' + filename)
else:
    custom_model.save(path + filename)

In [52]:
# Some small tests
number_of_words = str(len(custom_model.wv))
print('Number of words: ' + number_of_words)

Number of words: 6119


In [53]:
# Test to see what words are similar to pensioen
most_similar_to_pensioen = custom_model.wv.most_similar(positive='pensioen')
print("Words most similar to pensioen: ", most_similar_to_pensioen)

Words most similar to pensioen:  [('keuzepensioen', 0.7536688446998596), ('opgebouwde', 0.7353907227516174), ('brutoinkomen', 0.7202438712120056), ('uitvalt', 0.7131568193435669), ('afkoop', 0.7065716981887817), ('aowoverbrugging', 0.7033266425132751), ('ouderdomspensioen', 0.6991603374481201), ('rekenen', 0.6983091831207275), ('gedeelte', 0.6942805647850037), ('ruilt', 0.6924105882644653)]
